In [3]:
# 3) call get_data function . It would return list of following dataframe
# ['Product',
#  'Conditions_Condition_RowData',
#  'OfferCategory',
#  'Rewards_Reward_RewardTiers_RowData',
#  'Conditions_Condition_ConditionTiers_RowData',
#  'TerminalType',
#  'RowData',
#  'Location',
#  'ChargeBackDept',
#  'Terminals_RowData',
#  'Rewards_Reward_RowData',
#  'Rewards_Reward_Discounts_RowData']
# 4) all dataframes have joining keys to get desired sas dataset .
import datetime
import functools
print(datetime.datetime.now())
import copy
import pandas as pd
import io
import os
import sys
import logging as logger
import glob
from lxml import etree as et
import argparse
import subprocess
import re
import csv
from collections import OrderedDict

cols_dict={'offer_Customer_': ['cardtype', 'customergroup_extgroupid','customergroup_id', 'customergroup_name', 'id', 'promo_code'],\
           'offer_Product_': ['id',   'productgroup_desc',   'productgroup_id',   'productgroup_name',   'type',   'promo_code'],\
        'offer_Location_': ['id', 'locationgroup_id','locationgroup_name', 'name','promo_code'],\
           'offer_OfferCategory_': ['desc', 'id', 'promo_code'],\
                 'offer_Row_': ['advancedlimitid', 'autotransferable', 'cmoadeployrpt', 'cmoadeploystatus', 'cmoadeploysuccessdate', 'cmoarptdate', 'crmautodeployed', 'crmengineid', 'crmengineupdatelevel', 'crmrestricted', 'crmsendstatus', 'crmsendtoexport', 'createdbyadminid', 'createddate', 'deleted', 'deploydeferred',  'description', 'disabledisconnectedoffer', 'disabledoncfw', 'displayonwebkiosk', 'distperiod', 'distperiodlimit', 'distperiodvarid', 'employeefiltering', 'engineid', 'exporttoedw', 'extofferid', 'extofferid2', 'favorite', 'fromtemplate', 'highvalue', 'iwtranslevel', 'inboundcrmengineid', 'instantwin',  'istemplate', 'lastcrmsenddate', 'lastdistvarflush', 'lasttxdate', 'lastupdate', 'lastupdatedbyadminid', 'name', 'nonemployeesonly', 'numprizesallowed',  'numtiers', 'oddsofwinning', 'offercategoryid', 'offerid', 'offertypeid', 'prioritylevel', 'prodenddate', 'prodstartdate', 'purged', 'randomwinners',  'sharedlimitid', 'statusflag', 'testenddate', 'teststartdate', 'tiertypeid', 'updatelevel', 'visible', 'promo_code'],\
        'offer_Cons_Con_ConTiers_Row_': ['amtrequired','conditionid','pkid','tierlevel','promo_code'],\
        'offer_Rews_Rew_RewPoints_Row_': ['chargebackdeptid',   'discountableitemsonly',   'lastupdate',   'printlinetext',   'programid',   'rewardpointsid',   'webtext',   'promo_code'],\
        'offer_Cons_Con_Row_': ['crmastatusflag', 'conditionid','conditionorder','conditiontypeid', 'deleted', 'disallowedit1', 'disallowedit2', 'disallowedit3', 'disallowedit4', 'disallow_edit', 'discountableitemsonly', 'donotitemdistribute', 'excludedid', 'extseqnum', 'granttypeid', 'jointypeid', 'lastupdate', 'linkid', 'minorderitemsonly', 'offerid', 'pointsredeeminstant', 'pointsuseearnedvalue', 'pointsusenetvalue', 'qtyunittype', 'requiredfromtemplate', 'tcrmastatusflag', 'tiered', 'promo_code' ],\
        'offer_Rews_Rew_RewTiers_Row_': ['pkid','rewardamount','rewardid','tierlevel','promo_code'],\
        'offer_others': ['extgroupid','joinypeid','typeid','week','year'],\
         'offer_Rews_Rew_Row_': ['almultitrans',   'advancedlimitid',   'applytolimit',   'cmoastatusflag',   'deleted',   'disallowedit1',   'disallowedit2',   'disallowedit3',   'disallowedit4',   'disallowedit5',   'disallowedit6',   'disallowedit7',   'disallowedit8',   'disallowedit9',   'disallow_edit',   'donotitemdistribute',   'excludedprodgroupid',   'jointypeid',   'lastpromovarflush',   'lastupdate',   'linkid',   'offerid',   'productgroupid',   'promotetotranslevel',   'rewardamounttypeid',   'rewarddistlimitvarid',   'rewarddistperiod',   'rewardid',   'rewardlimit',   'rewardlimittypeid',   'rewardorder',   'rewardtypeid',   'sprepeatatoccur',   'sponsorid',   'tcrmastatusflag',   'tiered',   'triggerqty',   'usespecialpricing',   'promo_code']
           
        }
map_dict = {'./RowData':'offer_Row_','./Conditions/Condition/RowData':'offer_Cons_Con_Row_','./Conditions/Condition/ConditionTiers/RowData':'offer_Cons_Con_ConTiers_Row_'\
         ,'./Rewards/Reward/RowData':'offer_Rews_Rew_Row_','./Rewards/Reward/RewardTiers/RowData':'offer_Rews_Rew_RewTiers_Row_',\
          './Rewards/Reward/RewardPoints/RowData':'offer_Rews_Rew_RewPoints_Row_'}
mapp={'Customer':'offer_Customer_','Location':'offer_Location_','Product':'offer_Product_','OfferCategory':'offer_OfferCategory_'} 
reqlist = functools.reduce(list.__add__, cols_dict.values())

class GetXmlData(object):
    """
            Class to create csv by parsing the weekly xmls
    """

    def __init__(self,
                 xmlpath=None,
                 csvpath=None,
                 weekid=None):
        """
        Constructor for Parsing XML to create CSV class for raleys.
        Initializes the class variables
        """
        self.xmlpath = xmlpath
        self.csvpath = csvpath
        self.week_id = weekid
        self.row_data = './RowData'
        self.conditions = './Conditions/Condition/RowData'
        self.conditions_tier = "./Conditions/Condition/ConditionTiers/RowData"
        self.reward_rowdata = './Rewards/Reward/RowData'
        self.reward_rewpoints = './Rewards/Reward/RewardPoints/RowData'
        #self.reward_discount = './Rewards/Reward/Discounts/RowData'
        self.reward_rewardtiers = './Rewards/Reward/RewardTiers/RowData'
        #self.terminals = './Terminals/RowData'
        self.main_dict = {}
        
    count=0
    
    def create_csv(self, df_loc, my_file):
        """
        Method to create csv from dataframe
        """
        logger.debug(" Inside create_csv " + my_file)
        if os.path.isfile(my_file):
            with open(my_file, 'a') as fhandle:
                logger.debug("Appending CSV" + my_file)
                df_loc.to_csv(fhandle, header=False, index=False, encoding='utf-8',sep='|')
        else:
            logger.debug("Creating CSV" + my_file)
            df_loc.to_csv(my_file, index=False, encoding='utf-8', sep='|')
        return None

    def ensure_dir(self, file_path):
        """
            Ensures whether directory exists else creates it
        """
        logger.debug(" Inside ensure_dir " + file_path)
        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            logger.debug(" Creating Directory " + directory)
            os.makedirs(directory)
        return None
    
    def get_auxilary(self, node, node_lst, location):
        """
            Get the data from the tags attributes
        """
        logger.debug(" Inside get_auxilary ")
        logger.debug("Inside Auxilary method" + location)
        value = []
        child_list = node.getchildren()
        #print(child_list)  
        if not child_list:
            #print('11')
            node_list = ['ChargeBackDept','TerminalType','PointsProgram']  ###CustomerGroup
            if node.tag not in node_list:                           
                if node.tag not in self.main_dict.keys():
                                self.main_dict[node.tag] = []
                value = node.items()
                #print(value)
                #print(node_lst)
                value.extend(node_lst)
                value_dict = dict((x, y) for x, y in value)
                #print(value_dict)
                self.main_dict[node.tag].append(copy.copy(value_dict))
            print(self.main_dict)    
        else:
            for child in child_list:
                node_value = [(node.tag + "_" + ele[0], ele[1])  for ele in node.items() if ele[0].lower().strip() in reqlist]
                #print('00')
                #print(node_value)
                self.get_auxilary(child, node_value, location)
        return self.main_dict
    def get_offer(self, xpath, root, location):
        """
            Returns a pandas dataframe
        """

        logger.debug(" Inside get_offer " + location)
        data = [];one_row = {};missing_row = {};all_offer_row = OrderedDict();zz = {};app = []
        for row_data in root.findall(xpath):
            for i in cols_dict[map_dict[xpath]]:
                if i != 'promo_code':
                    for child in row_data.getchildren():
                        app.append(child.tag.lower().strip())
                if i not in app and i != 'promo_code':
                    missing_row[i] = 'not_available'
            for child in row_data.getchildren():
                if (child.tag.lower().strip() in reqlist):
                    one_row[child.tag] = child.text
            zz = one_row.copy()
            #print(zz)
            zz.update(missing_row)
            for kk in cols_dict[map_dict[xpath]]:
                alphlower = {k.lower(): v for k, v in zz.items()}
                if kk in alphlower.keys():
                    all_offer_row[kk] = alphlower[kk]
            data.append(copy.copy(all_offer_row))
        return pd.DataFrame(data)
    def get_data(self, location):
        """
            Returns the dictinary of values
        """

        logger.debug(" Inside get_data " + location)
        data1 = {}
        data2 = {}
        dict2 = OrderedDict()
        finallist1=[]
        finallist2=[]
        finaldict1={}
        finaldict2={}
        value_at_index=[]
        value_at={}
        dataD = OrderedDict()
        logger.debug("Open File" + location)
        with io.open(location, "r", encoding="utf-8") as myfile:
            data = myfile.read()
            test = et.fromstring(data.encode('utf-16'))
        dataD = self.get_auxilary(test.getchildren()[0], test, location)
        
        #od=data
       ##############################
        a=[]
        for k in dataD.keys():
            a.append(k)
        reqaux=mapp.keys()
        gog=[]#for customer,offer,location,product
        for i in a:
            if i in reqaux:
                gog.append(i)        
        j={}
        for i in gog:
            finallist1=[]
            for j in dataD[i]:
                ls1=cols_dict[mapp[i]]#req col
                dict2=OrderedDict()
                if 'promo_code' in ls1:
                    ls1.remove('promo_code')
                ls2=j.keys()
                ls2=[x.lower() for x in ls2]#data key
                value_at_index = j.values()#data value
                value_at=dict(zip(ls2, value_at_index)) #data dict   
                for x in ls1:
                    if x not in ls2:
                        dict2.update({x:'NA'})
                    else:        
                        dict2.update({x:value_at[x]})
                finallist1.append(dict2) 
            finaldict1={i:finallist1}
            finaldict2.update(finaldict1)
        ##############################'''
        for key in finaldict2.keys():
            data1[key] = pd.DataFrame.from_records(finaldict2[key])

        for tag in [self.row_data,
                    self.conditions,
                    self.conditions_tier,
                    self.reward_rowdata,
                    #self.reward_discount,
                    self.reward_rewardtiers,
                   self.reward_rewpoints
                   #self.terminals
                   ]:
            key = tag.replace("/", "_")
            key = key.replace("._", "")
            logger.debug(" Inside Offer Function " + location)
            data2[key] = self.get_offer(tag, test.getchildren()[1], location)

        data2 = {k:data2[k] for k in data2.keys() if len(data2[k]) > 0 }
        #print('9999999999999999999999')
        #print(data1.items())
        cv1 = data1.copy()
        cv2 = data2.copy()
        return dict(list(cv1.items()) + list(cv2.items()))
    def get_filename(self, key):
        """
            Returns the concised filename of the csv to be created
        """
        logger.debug(" Inside get_filename " + key)
        l_replace_dict = {"Condition":"Con", "Reward":"Rew", "RowData":"Row", "Discounts":"Dis"}
        for pattern, rep in l_replace_dict.items():
            key = key.replace(pattern, rep)
        return "offer_" + key

    def run_xml(self, filename, pattern="Offer", file_ext=".xml", code='Promo_Code'):
        """
            Returns None
        """
        logger.debug(" Inside run_xml " + filename )
        l_promo_code = filename.split(pattern, 1)[1].split(file_ext, 1)[0]

        logger.info(" Started Reading File" + filename)
        print (filename)
        print ("Reading xml file")
        l_data = self.get_data(filename)
        for k in l_data.keys():
            try:
                df_loc = l_data[k]
                df_loc[code] = l_promo_code
            except:
                print ('KeyError - reason')
            csv_file = self.csvpath + '/{DimData}_{dirname}.csv'.format(\
                                                                        DimData=self.get_filename(k),
                                                                        dirname=self.week_id)
            self.ensure_dir(csv_file)
            logger.info(" Started Creating CSV File : " + csv_file)
            self.create_csv(df_loc, csv_file)
        logger.info("***** Parsing File Completed : ****" + filename)
        return None

if __name__ == '__main__':

    XML_PATH = 'C:\\Users\\vprasad\*'
    CSV_PATH = 'C:\\Users\\vprasad\\'
    #week_id = subprocess.check_output(['hdfs dfs -text /raleys/tmp/nextfisweektoprocess.state/part-00000.gz'], shell=True).split('=')[-1][:6]
    week_id = 201839
    count=0
    print ("Unzipping All the Files")
    os.system('gunzip -f ' +XML_PATH+'Offer*xml.gz')

    logpath = CSV_PATH + 'xml_csv_log.log'
    logger.basicConfig(filename=logpath, level=logger.INFO)
    l_files_list = glob.glob(XML_PATH + 'Offer*.xml')
    getxmldata_obj = dict()
    logger.info("Processing the xmls ")

    for filename in l_files_list:
        getxmldata_obj[filename] = GetXmlData(xmlpath=XML_PATH,
                                csvpath=CSV_PATH,
                                weekid=week_id)
        getxmldata_obj[filename].run_xml(filename)

    csv_file_list = glob.glob(CSV_PATH+'/offer_*.csv')
    print ("Zipping the files")
    os.system('gzip '+ CSV_PATH +'/offer*.csv')
    print(datetime.datetime.now())

        #Added to move xml files from inbound to a directory of that week
        #logger.info("Moving the xml files to nfs archive location : /nfs/data_team/raleys/offer_xml_<fis_week_id>/")
        #file_path = '/nfs/data_team/raleys/offer_xml_{}/*'.format(week_id)
        #directory = os.path.dirname(file_path)
        #if not os.path.exists(directory):
        #   os.makedirs(directory)
        #subprocess.call('mv /raleys/datalake/inbound/Offer*.xml /nfs/data_team/raleys/offer_xml_{}/'.format(week_id),shell=True)


2020-02-22 12:46:45.046567
Unzipping All the Files
C:\Users\vprasad\Offer1377504.xml
Reading xml file
[]
[('ProductGroup_ID', '202511'), ('ProductGroup_Name', 'eCart Delivery Fee'), ('ProductGroup_Desc', '')]
{'Product': [{'ID': '00000000051613', 'Type': '1', 'ProductGroup_ID': '202511', 'ProductGroup_Name': 'eCart Delivery Fee', 'ProductGroup_Desc': ''}]}
[]
{'Product': [{'ID': '00000000051613', 'Type': '1', 'ProductGroup_ID': '202511', 'ProductGroup_Name': 'eCart Delivery Fee', 'ProductGroup_Desc': ''}], 'CustomerGroup': [{'ID': '337215', 'Name': 'eCart - RSC Free Delivery 2 Times', 'ExtGroupID': '', 'IsOptInGroup': 'false'}]}
[]
{'Product': [{'ID': '00000000051613', 'Type': '1', 'ProductGroup_ID': '202511', 'ProductGroup_Name': 'eCart Delivery Fee', 'ProductGroup_Desc': ''}], 'CustomerGroup': [{'ID': '337215', 'Name': 'eCart - RSC Free Delivery 2 Times', 'ExtGroupID': '', 'IsOptInGroup': 'false'}]}
[]
{'Product': [{'ID': '00000000051613', 'Type': '1', 'ProductGroup_ID': '202511', 'P

2020-02-22 12:46:45.295344


In [11]:
from functools import reduce
import operator
ls = [1,4,5]
reqlist = reduce(lambda x,y:x+y,ls)
let = reduce(operator.mul,ls)

In [12]:
let

20

In [71]:
from lxml import etree as et
XML_PATH = 'D:\\Dunhumby\\ab\Airflow\\XML Parser\\input\Offer1377504.xml'
with io.open(XML_PATH,'r') as xml:
    data = xml.read()#string
    test = et.fromstring(data.encode('utf-16'))
    print(test.getchildren())

[<Element Auxiliary at 0x1e024fe70c8>, <Element Offer at 0x1e023eddcc8>]


In [85]:
print(test.getchildren()[1].items())

[('Id', '1377504')]


In [81]:
for i,j in test[1].items():
    print(i)

Id


In [82]:
test[1].items()

[('Id', '1377504')]

In [ ]:
lxml->etree
fromstring
encode
test-->parent
test.items()
test[1/0]
test.getchildren()
test[1].items()


In [100]:
for i in test[0].getchildren():
    print(i.tag)
    print(i.items())

ProductGroup
[('ID', '202511'), ('Name', 'eCart Delivery Fee'), ('Desc', '')]
CustomerGroup
[('ID', '337215'), ('Name', 'eCart - RSC Free Delivery 2 Times'), ('ExtGroupID', ''), ('IsOptInGroup', 'false')]
TerminalType
[('ID', '1'), ('Name', 'All CM Terminals'), ('Desc', 'All CM Terminals'), ('ExtTerminalCode', '1'), ('LayoutID', '0'), ('EngineID', '0'), ('FuelProcessing', 'False'), ('AnyTerminal', 'True'), ('SpecificPromosOnly', '0')]
ChargeBackDept
[('ID', '0'), ('Name', "Item's Department"), ('ExternalId', '0')]
OfferCategory
[('ID', '7'), ('Desc', 'LT: Long Term Offers')]
LocationGroup
[('ID', '2'), ('Name', 'Test Stores (Now 140)')]
LocationGroup
[('ID', '102'), ('Name', '0 - Something Extra CRM Stores')]


In [103]:
for i in test[0].getchildren():
    #print(i.tag)
    for j in i.items():
        print(j[0]+j[1])

ID202511
NameeCart Delivery Fee
Desc
ID337215
NameeCart - RSC Free Delivery 2 Times
ExtGroupID
IsOptInGroupfalse
ID1
NameAll CM Terminals
DescAll CM Terminals
ExtTerminalCode1
LayoutID0
EngineID0
FuelProcessingFalse
AnyTerminalTrue
SpecificPromosOnly0
ID0
NameItem's Department
ExternalId0
ID7
DescLT: Long Term Offers
ID2
NameTest Stores (Now 140)
ID102
Name0 - Something Extra CRM Stores


In [112]:
node=test
print(node)
value = []
child_list = node.getchildren()
print(child_list)
for child in child_list:
    node_value = [(node.tag + "_" + ele[0], ele[1])  for ele in node.items() if ele[0].lower().strip() in reqlist]

<Element Auxiliary at 0x1e024f85f88>
[<Element ProductGroup at 0x1e0246d2fc8>, <Element CustomerGroup at 0x1e0250eca88>, <Element TerminalType at 0x1e023a76608>, <Element ChargeBackDept at 0x1e024705108>, <Element OfferCategory at 0x1e024de0c88>, <Element LocationGroup at 0x1e024de0f48>, <Element LocationGroup at 0x1e024de0848>]


In [111]:
node_value

[]

In [104]:
id1 = [('Id', '1377504'),('ProductGroup_Name', 'eCart Delivery Fee')]
id2 = [('Idd', '1377503'),('ProductGroup_Namee', 'Cart Delivery Fee')]
id1.extend(id2)
value_dict = dict((x, y) for x, y in id1)

In [105]:
id1

[('Id', '1377504'),
 ('ProductGroup_Name', 'eCart Delivery Fee'),
 ('Idd', '1377503'),
 ('ProductGroup_Namee', 'Cart Delivery Fee')]

In [106]:
value_dict

{'Id': '1377504',
 'ProductGroup_Name': 'eCart Delivery Fee',
 'Idd': '1377503',
 'ProductGroup_Namee': 'Cart Delivery Fee'}

In [107]:
id1 = [('ID', '00000000051613'), ('Type', '1')]
id2 = [('ProductGroup_ID', '202511'), ('ProductGroup_Name', 'eCart Delivery Fee')]
id1.extend(id2)
value_dict = dict((x, y) for x, y in id1)

In [108]:
id1

[('ID', '00000000051613'),
 ('Type', '1'),
 ('ProductGroup_ID', '202511'),
 ('ProductGroup_Name', 'eCart Delivery Fee')]

In [109]:
value_dict

{'ID': '00000000051613',
 'Type': '1',
 'ProductGroup_ID': '202511',
 'ProductGroup_Name': 'eCart Delivery Fee'}

In [110]:
vd = {'ID': '0000000001613'}

In [112]:
value_dict['ID'].append(copy.copy(vd))

AttributeError: 'str' object has no attribute 'append'

In [115]:
ad= {}
ad['ID']=[]

In [118]:
ad['ID'].append(copy.copy(value_dict))

In [119]:
ad

{'ID': [{'ID': '0000000001613'},
  {'ID': '00000000051613',
   'Type': '1',
   'ProductGroup_ID': '202511',
   'ProductGroup_Name': 'eCart Delivery Fee'}]}

In [136]:
a = {1: [1,2,3]}
b=a

In [141]:
b=copy.deepcopy(a)

In [142]:
b

{1: [1, 2, 3]}

In [135]:
b

{1: [1, 2, 3, 34]}